<a href="https://colab.research.google.com/github/heraclex12/FER-ResidualMaskingNetwork/blob/master/FER2013_ResNet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings

from keras.layers import Input
from keras import layers
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import AveragePooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import BatchNormalization, Lambda, Add, Multiply
from keras.models import Model
from keras.callbacks import EarlyStopping, TensorBoard, ReduceLROnPlateau, ModelCheckpoint
from keras import backend as K
from keras.engine.topology import get_source_inputs
from keras.utils import layer_utils, to_categorical
from keras.utils.data_utils import get_file
from tensorflow.image import resize
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
WEIGHTS_PATH = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.2/resnet50_weights_tf_dim_ordering_tf_kernels.h5'
WEIGHTS_PATH_NO_TOP = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.2/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

Using TensorFlow backend.


Some solution for convert 1channel into 3 channels

First, as below, using numpy repeat built-in function: x = np.repeat(x, 3, -1)

Second, I haven't try it yet, but I saw it in kaggle, just try.

- <code>img_input = Input(shape=(img_size_target,img_size_target,1)) </br>
img_conc = Concatenate()([img_input, img_input, img_input])   </code>

Third, it depend on which library you use, but main step is you must exclude first convolutional layer'pretrained model. in Keras, you put exclude=['conv1'] on load_weights statement and initialize random weight (better than, average it) for it; in Pytorch, you get conv1 weight ouput and sum it into 1-dimension

Fourth, you can use ImageDataGenerator in Keras to load grayscale as RGB

### RESNET50 model

In [ ]:

def identity_block(input_tensor, kernel_size, filters, stage, block):
    filters1, filters2, filters3 = filters
    bn_axis = 3
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size,
               padding='same', name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    x = Add()([x, input_tensor])
    x = Activation('relu')(x)
    return x


def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):

    filters1, filters2, filters3 = filters
    bn_axis = 3
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), strides=strides,
               name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, padding='same',
               name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    shortcut = Conv2D(filters3, (1, 1), strides=strides,
                      name=conv_name_base + '1')(input_tensor)
    shortcut = BatchNormalization(axis=bn_axis, name=bn_name_base + '1')(shortcut)

    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

In [ ]:
def ResNet50(weights='imagenet',
             input_tensor=None, input_shape=None,
             pooling=None,
             classes=1000):
  
    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor

    bn_axis = 3

    x = Lambda(lambda img : tf.image.resize(img, (224, 224)))(img_input)
    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same', name='conv1')(x)
    x = BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding="same")(x)

    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')

    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f')

    x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')

        # AVGPOOL
    x = AveragePooling2D(pool_size=(2,2), padding='same')(x)

    # Output layer
    x = Flatten()(x)
    x = Dense(classes, activation='softmax', name='fc' + str(classes))(x)

    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input
    # Create model.
    model = Model(inputs, x, name='resnet50')

    # load weights
    weights_path = get_file('resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5',
                                    WEIGHTS_PATH_NO_TOP,
                                    cache_subdir='models',
                                    md5_hash='a268eb855778b3df3c7506639542a6af')
    # weights_path = "drive/My Drive/fer2013_data/models/weights.h5"
    model.load_weights(weights_path,by_name=True)
    return model

### LOAD DATA

In [ ]:
training_set = pd.read_csv("drive/My Drive/fer2013_data/train.csv")
test_set = pd.read_csv("drive/My Drive/fer2013_data/test.csv")
validation_set = pd.read_csv("drive/My Drive/fer2013_data/val.csv")

In [ ]:
training_labels = to_categorical(training_set['emotion'])
test_labels = to_categorical(test_set['emotion'])
validation_labels = to_categorical(validation_set['emotion'])

In [ ]:
training_pixels = training_set['pixels'].str.split().tolist()
training_pixels = np.array(training_pixels)
training_pixels = training_pixels.reshape(-1, 48, 48, 1)
training_pixels = np.repeat(training_pixels, 3, -1)        # convert to 3channels
training_pixels = training_pixels.astype("float32") / 255

test_pixels = test_set['pixels'].str.split().tolist()
test_pixels = np.array(test_pixels)
test_pixels = test_pixels.reshape(-1, 48, 48, 1)
test_pixels = np.repeat(test_pixels, 3, -1)        # convert to 3channels
test_pixels = test_pixels.astype("float32") / 255

validation_pixels = validation_set['pixels'].str.split().tolist()
validation_pixels = np.array(validation_pixels)
validation_pixels = validation_pixels.reshape(-1, 48, 48, 1)
validation_pixels = np.repeat(validation_pixels, 3, -1)        # convert to 3channels
validation_pixels = validation_pixels.astype("float32") / 255

In [ ]:
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, min_lr = 1e-8)

early_stopper = EarlyStopping(monitor='val_loss', min_delta=0, patience=8, verbose=1, mode='auto')

tensorBoard = TensorBoard(log_dir='drive/My Drive/fer2013_data/logs')

checkpointer = ModelCheckpoint("drive/My Drive/fer2013_data/models/resnet50_with_pretrained_model.h5", monitor='val_loss', verbose=1, save_best_only=True)

### Freeze and training

In [ ]:
checkpointer = ModelCheckpoint("drive/My Drive/fer2013_data/models/resnet50_with_pretrained_model.h5", monitor='val_accuracy', verbose=1, save_best_only=True)

model = ResNet50(input_shape=(48, 48, 3), classes=7)

for layer in model.layers[:-1]:
  layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, min_lr = 1e-8)
model.fit(training_pixels, training_labels, batch_size = 48, epochs=10, validation_data= (validation_pixels, validation_labels),
          callbacks=[lr_reducer, checkpointer])

model.load_weights("drive/My Drive/fer2013_data/models/resnet50_with_pretrained_model.h5")
score, acc = model.evaluate(validation_pixels, validation_labels, verbose=0)
print("Accuracy:", acc)
print("Score:", score)
score, acc = model.evaluate(test_pixels, test_labels, verbose=0)
print("Accuracy:", acc)
print("Score:", score)

Train on 28709 samples, validate on 3589 samples
Epoch 1/10
28709/28709 [==============================] - 89s 3ms/step - loss: 3.4183 - accuracy: 0.4403 - val_loss: 4.5966 - val_accuracy: 0.1301

Epoch 00001: val_accuracy improved from -inf to 0.13012, saving model to drive/My Drive/fer2013_data/models/resnet50_with_pretrained_model.h5
Epoch 2/10
28709/28709 [==============================] - 85s 3ms/step - loss: 2.1430 - accuracy: 0.6246 - val_loss: 6.1527 - val_accuracy: 0.0156

Epoch 00002: val_accuracy did not improve from 0.13012
Epoch 3/10
28709/28709 [==============================] - 85s 3ms/step - loss: 1.5405 - accuracy: 0.7123 - val_loss: 6.5707 - val_accuracy: 0.1934

Epoch 00003: val_accuracy improved from 0.13012 to 0.19337, saving model to drive/My Drive/fer2013_data/models/resnet50_with_pretrained_model.h5
Epoch 4/10
28709/28709 [==============================] - 86s 3ms/step - loss: 1.2256 - accuracy: 0.7640 - val_loss: 9.1252 - val_accuracy: 0.1301

Epoch 00004: Redu

### Unfreeze and training

In [ ]:
checkpointer = ModelCheckpoint("drive/My Drive/fer2013_data/models/resnet50_with_pretrained_model.h5", monitor='val_accuracy', verbose=1, save_best_only=True)

for layer in model.layers[:-1]:
  layer.trainable = True

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, min_lr = 1e-8)

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(training_pixels, training_labels, batch_size = 48, epochs=50, validation_data= (validation_pixels, validation_labels),
          callbacks=[lr_reducer, checkpointer])

model.load_weights("drive/My Drive/fer2013_data/models/resnet50_with_pretrained_model.h5")
score, acc = model.evaluate(test_pixels, test_labels, verbose=0)
print("Accuracy:", acc)
print("Score:", score)

Train on 28709 samples, validate on 3589 samples
Epoch 1/50
28709/28709 [==============================] - 315s 11ms/step - loss: 1.8522 - accuracy: 0.4743 - val_loss: 1.8918 - val_accuracy: 0.1808

Epoch 00001: val_accuracy improved from -inf to 0.18083, saving model to drive/My Drive/fer2013_data/models/resnet50_with_pretrained_model.h5
Epoch 2/50
28709/28709 [==============================] - 299s 10ms/step - loss: 1.2270 - accuracy: 0.5600 - val_loss: 1.5841 - val_accuracy: 0.3670

Epoch 00002: val_accuracy improved from 0.18083 to 0.36695, saving model to drive/My Drive/fer2013_data/models/resnet50_with_pretrained_model.h5
Epoch 3/50
28709/28709 [==============================] - 299s 10ms/step - loss: 1.3152 - accuracy: 0.5570 - val_loss: 3.1884 - val_accuracy: 0.1900

Epoch 00003: val_accuracy did not improve from 0.36695
Epoch 4/50
28709/28709 [==============================] - 299s 10ms/step - loss: 1.3560 - accuracy: 0.5332 - val_loss: 1.1913 - val_accuracy: 0.5514

Epoch 000

In [ ]:
# model.layers[1].weights[0][0][0][0]